In [2]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from Bio import Phylo
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import random
from multiprocessing import Process, Manager
from multiprocessing import Pool

In [ ]:
! aws s3 sync s3://florencia-velez/working/20220927_HGM_signalP/signalp_analysis/ /home/ec2-user/20220927_HGM_signalP/signalp_analysis/
! aws s3 cp s3://florencia-velez/working/20190504_HGM_prodigal/IGG_species_info_23790.tsv /home/ec2-user/20190504_HGM_prodigal/IGG_species_info_23790.tsv
! aws s3 cp s3://florencia-velez/working/20190504_HGM_prodigal/IGG_genome_info_206581.tsv /home/ec2-user/20190504_HGM_prodigal/IGG_genome_info_206581.tsv
! aws s3 sync s3://florencia-velez/working/20221003_HGM_Parks_tmhmm/HGM_tmhmm_out/output/ /home/ec2-user/20221003_HGM_Parks_tmhmm/HGM_tmhmm_out/output/

In [4]:
#Import metadata for HGM MAGs
HGM_species = pd.read_csv('/home/ec2-user/20190504_HGM_prodigal/IGG_species_info_23790.tsv', sep='\t')
HGM_genomes = pd.read_csv('/home/ec2-user/20190504_HGM_prodigal/IGG_genome_info_206581.tsv', sep='\t')

HGM_species_hqHGM_species = HGM_genomes[(HGM_genomes['repository'] == 'HGM') & (HGM_genomes['quality_level'] == 'high')]['species_id'].tolist()

HGM_species_hqHGM = HGM_species[HGM_species['species_id'].isin(HGM_species_hqHGM_species)]

/home/ec2-user/bin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [101]:
os.mkdir('ipynb_out')

```
! cat /home/ec2-user/20221003_HGM_Parks_tmhmm/HGM_tmhmm_out/output/*.out | fgrep "Number of predicted TMHs:" > ~/20221005_metasecretome_notebooks/ipynb_out/HGM_tmhmm_out_concat.txt
HGM_tm_pd = pd.read_csv("~/20221005_metasecretome_notebooks/ipynb_out/HGM_tmhmm_out_concat.txt", header=None, names=['orfid','Number of TM'],
            delim_whitespace=True, usecols=[1,6])
    
HGM_orfids_noTM = HGM_tm_pd[HGM_tm_pd['Number of TM'] == 0]['orfid'].unique().tolist()
# 1 million centroids of large clusters have zero TM domains 
len(HGM_orfids_noTM)
#1047268
```

In [111]:
# Add signalp prediction to large cluster ORFs
def signalp_orf_prediction():
    signalp_class = dict()
    ## Add membrane type for signalp
    for species_id in HGM_species_hqHGM['species_id']:
        tax = HGM_species_hqHGM[HGM_species_hqHGM['species_id'] == species_id]['gtdb_taxonomy'].tolist()
        if len(tax) == 1:
            tax = tax[0].split(';')[1].replace('p__','')
            if tax in ['Actinobacteria', 'Firmicutes']:
                sptype = 'gram-positive'
            elif tax in ['Bacteroidetes', 'Cyanobacteria', 'Desulfobacteraeota_A', 'Elusimicrobia', 'Epsilonbacteraeota',
                        'Fibrobacteres', 'Fusobacteria', 'Proteobacteria', 'Spirochaetes', 'Verrucomicrobia']:
                sptype = 'gram-negative'
            elif tax in ['Thermoplasmataeota','Euryarchaeota']:
                sptype = 'archaea'
            signalp_class[species_id] = sptype
            
    HGM_species_hqHGM_sptype = HGM_species_hqHGM.set_index('species_id').join(pd.DataFrame(pd.Series(signalp_class, name='signalp_type')))
    HGM_species_hqHGM_sptype.to_csv('ipynb_out/HGM_species_hqHGM_sptype.tsv', sep='\t')
    
    HGM_grampos_pred = pd.read_csv('~/20220927_HGM_signalP/signalp_analysis/HGM_all-grampos_summary.tsv', skiprows=1, sep='\t',
                              header=None, comment='#', names=['# ID','Prediction','SP(Sec/SPI)','TAT(Tat/SPI)','LIPO(Sec/SPII)','OTHER','CS Position'])
    HGM_gramneg_pred = pd.read_csv('~/20220927_HGM_signalP/signalp_analysis/HGM_all-gramneg_summary.tsv', skiprows=1, sep='\t',
                                  header=None, comment='#', names=['# ID','Prediction','SP(Sec/SPI)','TAT(Tat/SPI)','LIPO(Sec/SPII)','OTHER','CS Position'])

    HGM_grampos_pred['genome_id'] = ['_'.join(orfid.split('_')[:2]) for orfid in HGM_grampos_pred['# ID']]
    HGM_gramneg_pred['genome_id'] = ['_'.join(orfid.split('_')[:2]) for orfid in HGM_gramneg_pred['# ID']]

    
    for col in ['SP(Sec/SPI)','TAT(Tat/SPI)','LIPO(Sec/SPII)']:
        #Turn from string to float
        HGM_gramneg_pred[col] = [float(val) for val in HGM_gramneg_pred[col]]
        HGM_grampos_pred[col] = [float(val) for val in HGM_grampos_pred[col]]

    
    #Limit to correct gram classification, leave archaea out. Only consider Sec and Tat secretion tags, not lipoproteins
    #secreted_condition_pos = (HGM_grampos_pred['SP(Sec/SPI)'] > 0.9) | (HGM_grampos_pred['TAT(Tat/SPI)'] > 0.9) | (HGM_grampos_pred['LIPO(Sec/SPII)'] > 0.9)
    secreted_condition_pos = (HGM_grampos_pred['SP(Sec/SPI)'] > 0.9) | (HGM_grampos_pred['TAT(Tat/SPI)'] > 0.9)
    grampos_species = HGM_species_hqHGM_sptype[HGM_species_hqHGM_sptype['signalp_type'] == 'gram-positive'].index.tolist()
    grampos_genomes = HGM_genomes[HGM_genomes['species_id'].isin(grampos_species)]['genome_id'].unique().tolist()

    HGM_grampos_pred = HGM_grampos_pred[(secreted_condition_pos) & (HGM_grampos_pred['genome_id'].isin(grampos_genomes))]

    #secreted_condition_neg = (HGM_gramneg_pred['SP(Sec/SPI)'] > 0.9) | (HGM_gramneg_pred['TAT(Tat/SPI)'] > 0.9) | (HGM_gramneg_pred['LIPO(Sec/SPII)'] > 0.9)
    secreted_condition_neg = (HGM_gramneg_pred['SP(Sec/SPI)'] > 0.9) | (HGM_gramneg_pred['TAT(Tat/SPI)'] > 0.9)
    gramneg_species = HGM_species_hqHGM_sptype[HGM_species_hqHGM_sptype['signalp_type'] == 'gram-negative'].index.tolist()
    gramneg_genomes = HGM_genomes[HGM_genomes['species_id'].isin(gramneg_species)]['genome_id'].unique().tolist()

    HGM_gramneg_pred = HGM_gramneg_pred[(secreted_condition_neg) & (HGM_gramneg_pred['genome_id'].isin(gramneg_genomes))]
    
    HGM_pred = dict()
    HGM_pred['grampos'] = HGM_grampos_pred
    HGM_pred['gramneg'] = HGM_gramneg_pred
    return(HGM_pred)

In [112]:
HGM_pred = signalp_orf_prediction()
HGM_grampos_pred = HGM_pred['grampos']
HGM_gramneg_pred = HGM_pred['gramneg']

In [113]:
HGM_grampos_pred[:5]

,# ID,Prediction,SP(Sec/SPI),TAT(Tat/SPI),LIPO(Sec/SPII),OTHER,CS Position,genome_id
553,ERS235556_24_k99_1632_50,SP(Sec/SPI),0.996039,0.000698,0.001085,0.002178,CS pos: 25-26. VFA-SE. Pr: 0.7901,ERS235556_24
590,ERS235509_25_k99_5401_10,SP(Sec/SPI),0.971652,0.003394,0.008064,0.016890,CS pos: 30-31. VEA-QN. Pr: 0.8572,ERS235509_25
886,ERS235528_45_k99_20365_10,SP(Sec/SPI),0.943362,0.002346,0.047495,0.006797,CS pos: 25-26. VYA-SD. Pr: 0.9328,ERS235528_45
999,ERS235509_25_k99_36532_3,SP(Sec/SPI),0.936414,0.002945,0.010553,0.050087,CS pos: 30-31. QRA-AD. Pr: 0.4839,ERS235509_25
1187,ERS235521_6_k99_51647_53,SP(Sec/SPI),0.971357,0.018323,0.004248,0.006072,CS pos: 28-29. VYA-AG. Pr: 0.8775,ERS235521_6


In [114]:
HGM_gramneg_pred[:5]

,# ID,Prediction,SP(Sec/SPI),TAT(Tat/SPI),LIPO(Sec/SPII),OTHER,CS Position,genome_id
259,ERS235517_65_k99_34625_6,TAT(Tat/SPI),0.000084,0.999780,0.000072,0.000064,CS pos: 33-34. ALA-ET. Pr: 0.9787,ERS235517_65
1338,ERS235558_17_k99_132405_17,SP(Sec/SPI),0.998455,0.000238,0.000723,0.000584,CS pos: 23-24. ALA-ED. Pr: 0.9951,ERS235558_17
1650,ERS235517_65_k99_147147_18,SP(Sec/SPI),0.947647,0.015036,0.010295,0.027022,CS pos: 27-28. AQA-EK. Pr: 0.8072,ERS235517_65
1741,ERS235558_17_k99_94759_14,SP(Sec/SPI),0.996951,0.001029,0.001420,0.000600,CS pos: 26-27. ALA-KI. Pr: 0.8768,ERS235558_17
1748,ERS235517_65_k99_28934_33,SP(Sec/SPI),0.998265,0.000610,0.000805,0.000319,CS pos: 25-26. AMA-KD. Pr: 0.7344,ERS235517_65


In [12]:
! aws s3 cp s3://florencia-velez/working/20220907_HGM_usearch/all_HGM_clusters.tar.gz /home/ec2-user/20220907_HGM_usearch/all_HGM_clusters.tar.gz
! aws s3 cp s3://florencia-velez/working/20220907_HGM_usearch/HGM_largeclustercentroids.oneline.fasta /home/ec2-user/20220907_HGM_usearch/HGM_largeclustercentroids.oneline.fasta

download: s3://florencia-velez/working/20220907_HGM_usearch/all_HGM_clusters.tar.gz to ../20220907_HGM_usearch/all_HGM_clusters.tar.gz


```
#Put all HGM proteins clustered at 95% amino acid identity in one text file
$ cd 20220907_HGM_usearch/
$ tar -xzf all_HGM_clusters.tar.gz
$ cd all_HGM_clusters/
$ find . -type f | xargs fgrep ">" > /home/ec2-user/20221005_metasecretome_notebooks/all_HGM_clusters_orfs.txt &
$ fgrep -c ">" /home/ec2-user/20221005_metasecretome_notebooks/all_HGM_clusters_orfs.txt 
61426555 #There are 61 million proteins in HGM
#Get only large secreted cluster proteins
$ fgrep ">" /home/ec2-user/20220907_HGM_usearch/HGM_largeclustercentroids.oneline.fasta | fgrep -w -f - /home/ec2-user/20221005_metasecretome_notebooks/all_HGM_clusters_orfs.txt | awk -F ':' '{print $1}' > /home/ec2-user/20221005_metasecretome_notebooks/HGM_largeclusterids.txt &
$ fgrep -w -f /home/ec2-user/20221005_metasecretome_notebooks/HGM_largeclusterids.txt /home/ec2-user/20221005_metasecretome_notebooks/all_HGM_clusters_orfs.txt > /home/ec2-user/20221005_metasecretome_notebooks/HGM_largecluster_orfs.txt &
53607501 #53 million of the proteins in HGM are in cluster with at least 5 member sequences
```

In [127]:
#Propagate signalp predictions from centroids to all members of large clusters
def HGM_clusterinfo_df():
    #Save HGM orfs from clusters >5 sequences to a dataframe
    HGM_largecluster_orfs = pd.read_csv('/home/ec2-user/20221005_metasecretome_notebooks/HGM_largecluster_orfs.txt', delimiter=':', header=None, names=['foldername','orfid'])
    HGM_largecluster_orfs['orfid'] = [orfid.replace('>','').split(' ')[0] for orfid in HGM_largecluster_orfs['orfid']]
    HGM_largecluster_orfs['clusterid'] = [foldername.replace('./all_HGM_','') for foldername in HGM_largecluster_orfs['foldername']]
    HGM_largecluster_orfs['genomeid'] = ['_'.join(orfid.split('_')[:2]) for orfid in HGM_largecluster_orfs['orfid']]  
    
    #Make list of ORFs from clusters >5 sequences that have a signal peptide
    #HGM_largeclustercentroids_mature_secreted = SeqIO.to_dict(SeqIO.parse('/home/ec2-user/20200202_HGM_Parks_signalp/signalp_analysis/high_confidence_mature_HGM.fasta','fasta'))
    #largeclus_secreted_orfids = list(HGM_largeclustercentroids_mature_secreted.keys())
    largeclus_secreted_orfids_nosigtag = HGM_grampos_pred['# ID'].unique().tolist() + HGM_gramneg_pred['# ID'].unique().tolist()
    #largeclus_secreted_orfids_nosigtag = list(set([orf.replace('_neg','').replace('_pos','') for orf in largeclus_secreted_orfids]))
    
    #Select for orfids that have no TM domains 
    HGM_tm_pd = pd.read_csv("~/20221005_metasecretome_notebooks/ipynb_out/HGM_tmhmm_out_concat.txt", header=None, names=['orfid','Number of TM'],
            delim_whitespace=True, usecols=[1,6])
    
    largeclus_orfids_notm = HGM_tm_pd[HGM_tm_pd['Number of TM'] == 0]['orfid'].unique().tolist()
    
    #Get orfids that have no TM and have a secretion tag
    largeclus_secreted_orfids_notm = list(set(largeclus_secreted_orfids_nosigtag) & set(largeclus_orfids_notm))
    
    #Mark as secreted all orfids belonging to cluster where representative is secreted
    HGM_largecluster_orfs['secreted'] = 'no'
    secreted_clusterids = HGM_largecluster_orfs[HGM_largecluster_orfs['orfid'].isin(largeclus_secreted_orfids_notm)]['clusterid']
    HGM_largecluster_orfs.loc[HGM_largecluster_orfs['clusterid'].isin(secreted_clusterids),'secreted'] = 'yes'
    #HGM_largecluster_orfs['secreted'].value_counts()
    
    HGM_largecluster_orfs.to_csv('ipynb_out/HGM_largecluster_orfs.tsv', sep='\t')

In [128]:
HGM_clusterinfo_df()

In [129]:
#Propagated new signalp results to clusters and output to tsv file
def propagate_sec_to_tsv():
    HGM_largecluster_orfs = pd.read_csv('ipynb_out/HGM_largecluster_orfs.tsv', sep='\t', index_col=0)
    HGM_largecluster_sec_orfs = HGM_largecluster_orfs[HGM_largecluster_orfs['secreted'] == 'yes'][['orfid','genomeid','clusterid']]

    #Add info about type of secretion
    HGM_largecluster_sec_orfs = HGM_largecluster_sec_orfs.set_index('orfid').join(HGM_gramneg_pred[['# ID','Prediction']].append(HGM_grampos_pred[['# ID','Prediction']]).set_index('# ID'))
    #Propagate from centroid to other members of cluster (assumption is that clusters are mainly in the same gram type)
    for sectype in HGM_largecluster_sec_orfs['Prediction'].unique():
        if sectype == sectype:
            secreted_clusterids = HGM_largecluster_sec_orfs[HGM_largecluster_sec_orfs['Prediction'] == sectype]['clusterid']
            HGM_largecluster_sec_orfs.loc[HGM_largecluster_sec_orfs['clusterid'].isin(secreted_clusterids), 'Prediction'] = sectype 
    #Add secretion prediction info to all HGM large cluster ORFs
    HGM_largecluster_orfs_secpred = HGM_largecluster_orfs.set_index('orfid').join(HGM_largecluster_sec_orfs['Prediction'])
    HGM_largecluster_orfs_secpred['Prediction'] = HGM_largecluster_orfs_secpred['Prediction'].fillna('Not_secreted')
    HGM_largecluster_orfs_secpred[['clusterid','genomeid','secreted','Prediction']].to_csv('ipynb_out/HGM_largecluster_orfs_secpred.tsv', sep='\t')


In [130]:
propagate_sec_to_tsv()

In [4]:
HGM_largecluster_orfs_secpred = pd.read_csv('ipynb_out/HGM_largecluster_orfs_secpred.tsv', sep='\t')
secreted_largecluster_orfs = HGM_largecluster_orfs_secpred[(HGM_largecluster_orfs_secpred['secreted'] == 'yes')].orfid.unique().tolist()


In [4]:
#Make a fasta of all HGM secreted ORFs (no TM) in large clusters
def get_aa_rec(orfid):
    faa_path = "/home/ec2-user/20220906_HGM_prodigal/prodigal_output_HGM/faa_output/"
    magid = '_'.join(orfid.split('_')[:2])
    orfrec = 0
    with open(faa_path + magid + ".faa") as handle:
        recs = SeqIO.parse(handle, 'fasta')
        for rec in recs:
            if rec.id == orfid:
                orfrec = rec
                #seq = str(rec.seq).replace('*','')
    return(orfrec)

def get_sec_aa_recs_from_one_mag(magid):
    faa_path = "/home/ec2-user/20220906_HGM_prodigal/prodigal_output_HGM/faa_output/"
    sec_aa_orfids_in_mag = HGM_largecluster_orfs_secpred[(HGM_largecluster_orfs_secpred['secreted'] == 'yes') & (HGM_largecluster_orfs_secpred['genomeid'] == magid)].orfid.unique().tolist()
    recs_in_mag = []
    with open(faa_path + magid + ".faa") as handle:
        recs = SeqIO.parse(handle, 'fasta')
        for rec in recs:
            if rec.id in sec_aa_orfids_in_mag:
                recs_in_mag.append(rec)
                #seq = str(rec.seq).replace('*','')
    return(recs_in_mag)

def write_sec_fasta(magid):
    reclist = get_sec_aa_recs_from_one_mag(magid)
    SeqIO.write(reclist, "ipynb_out/secreted_orfs_fastas/HGM_secreted_large_cluster_" + magid +".faa", "fasta")

In [5]:
HGM_largecluster_orfs_secpred[:4]

,orfid,clusterid,genomeid,secreted,Prediction
0,ERS235496_24_k99_12476_24,clusters0,ERS235496_24,no,Not_secreted
1,ERS235501_56_k99_98957_2,clusters0,ERS235501_56,no,Not_secreted
2,ERS235504_6_k99_4431_112,clusters0,ERS235504_6,no,Not_secreted
3,ERS235508_18_k99_12042_6,clusters0,ERS235508_18,no,Not_secreted


In [6]:
HGM_largecluster_orfs_secpred.secreted.value_counts()

no     51979543
yes     1627958
Name: secreted, dtype: int64

In [7]:
HGM_largecluster_orfs_secpred.Prediction.value_counts()

Not_secreted    51979543
SP(Sec/SPI)      1552495
TAT(Tat/SPI)       75463
Name: Prediction, dtype: int64

In [8]:
#There are about 1.6 million secreted ORFs in large clusters
len(secreted_largecluster_orfs)

1627958

In [16]:
os.mkdir("ipynb_out/secreted_orfs_fastas/")
magids_with_large_cluster_sec_orfs = HGM_largecluster_orfs_secpred[(HGM_largecluster_orfs_secpred['secreted'] == 'yes')].genomeid.unique()
with Pool(16) as p:
    print(p.map(write_sec_fasta, magids_with_large_cluster_sec_orfs))
#Concatenate MAG secreted orf fastas into one fasta with all secreted orfs in HGM
! cat ipynb_out/secreted_orfs_fastas/*.faa > HGM_secreted_orfs.faa

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [24]:
! ls -lth HGM_secreted_orfs.faa

-rw-rw-r-- 1 ec2-user ec2-user 966M Oct 11 21:17 HGM_secreted_orfs.faa


In [19]:
#Make a fasta of representative MAG HGM secreted ORFs (no TM) in large clusters

! aws s3 cp s3://florencia-velez/working/20210519_signalp_analysis/ipynb_out/representative_hgm_mags.csv /home/ec2-user/20210519_signalp_analysis/ipynb_out/representative_hgm_mags.csv
representative_mags = pd.read_csv('/home/ec2-user/20210519_signalp_analysis/ipynb_out/representative_hgm_mags.csv')['genomeid'].unique().tolist()

secreted_largecluster_orfs_repmags = HGM_largecluster_orfs_secpred[(HGM_largecluster_orfs_secpred['secreted'] == 'yes') & (HGM_largecluster_orfs_secpred['genomeid'].isin(representative_mags))].orfid.unique().tolist()

download: s3://florencia-velez/working/20210519_signalp_analysis/ipynb_out/representative_hgm_mags.csv to ../20210519_signalp_analysis/ipynb_out/representative_hgm_mags.csv


In [21]:
#There are 34,384 secreted ORFs in large clusters of representative MAGs
len(secreted_largecluster_orfs_repmags)

34384

In [30]:
#repmags_with_secreted_orfs = HGM_largecluster_orfs_secpred[(HGM_largecluster_orfs_secpred['secreted'] == 'yes') & (HGM_largecluster_orfs_secpred['genomeid'].isin(representative_mags))].genomeid.unique().tolist()
#for mag in repmags_with_secreted_orfs:
HGM_secreted_orfs_seq_dict = dict()
with open("HGM_secreted_orfs.faa") as handle:
    HGM_secreted_orfs_seq_dict = SeqIO.to_dict(SeqIO.parse(handle, "fasta"))
    
repmag_secorf_seq_dict = { orf: HGM_secreted_orfs_seq_dict[orf] for orf in secreted_largecluster_orfs_repmags }

In [40]:
SeqIO.write(list(repmag_secorf_seq_dict.values()), "ipynb_out/HGM_representative_MAG_secreted_orfs.faa", "fasta")

34384